# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [70]:
# import libraries
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import warnings
import pickle

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import RidgeClassifierCV
from sklearn.metrics import classification_report, hamming_loss, recall_score, make_scorer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, HashingVectorizer

#nltk for tokenizing/lemmatizing
import nltk
nltk.download(['punkt','wordnet'])

warnings.filterwarnings(action='once')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Ben\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Ben\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [49]:
# load data from database
engine = create_engine('sqlite:///textmessages.db')
df = pd.read_sql_table('MessageTable',con=engine)
df_orig_rows = df.shape[0]

In [50]:
#check for remaining null values
df[df.isnull().any(axis=1)]

,id,message,genre,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report


In [51]:
#split into features and labels
X = df['message']
y = df.iloc[:,3:]

**Check feature/label split**

In [22]:
X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

In [23]:
y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
print(y.shape, X.shape)

(26028, 36) (26028,)


### 2. Write a tokenization function to process your text data

In [25]:
#this function will be used later as the tokenizer param in the CountVectorizer 
def tokenize(text):
    #split into tokens
    tokens = nltk.word_tokenize(text)
    
    #get to the ROOT of the words (lemmatize)
    lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()
    
    clean_tokens = []
    for token in tokens:
        clean_token = lemmatizer.lemmatize(token, pos='v').lower()
        clean_tokens.append(clean_token)
    
    return clean_tokens

### 3. Build a machine learning pipeline
- You'll find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.
- List of classifiers that support multi-label classification can be found [here](https://scikit-learn.org/stable/modules/multiclass.html)

In [9]:
#pipeline
pipeline = Pipeline(
    [('vectorizer',CountVectorizer(tokenizer=tokenize)),
     ('tfidf',TfidfTransformer()),
     ('clf',RandomForestClassifier(random_state=42))]
)

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25, random_state=42)

In [11]:
%time pipeline.fit(X_train, y_train)

Wall time: 30 s


Pipeline(memory=None,
     steps=[('vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
       ...stimators=10, n_jobs=1,
            oob_score=False, random_state=42, verbose=0, warm_start=False))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

Found this useful discussion of specific average metrics [here](http://rushdishams.blogspot.com/2011/08/micro-and-macro-average-of-precision.html), and [here](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.recall_score.html) are the docs on recall_scorer (in which there is discussion of keyword 'average')

# Base Model Classification Metrics (RandomForestClassifier)

In [20]:
y_pred = pipeline.predict(X_test)
target_names = y.columns
print(classification_report(y_test, y_pred, target_names=target_names))

                        precision    recall  f1-score   support

               related       0.83      0.90      0.87      4938
               request       0.85      0.37      0.52      1104
                 offer       0.00      0.00      0.00        26
           aid_related       0.76      0.42      0.54      2714
          medical_help       0.57      0.01      0.03       542
      medical_products       0.86      0.03      0.07       347
     search_and_rescue       1.00      0.02      0.03       192
              security       0.50      0.01      0.01       133
              military       0.57      0.02      0.03       225
           child_alone       0.00      0.00      0.00         0
                 water       0.88      0.08      0.15       430
                  food       0.85      0.22      0.35       731
               shelter       0.79      0.14      0.24       598
              clothing       0.59      0.10      0.17       101
                 money       0.75      

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [21]:
hamming_loss(y_test,y_pred)

0.06176254631764083

### 6. Improve your model
Use grid search to find better parameters. 

In [54]:
# pipeline.get_params()

# parameters = {'clf__estimator__n_estimators':[10,50,100], 'vectorizer__stop_words': ['english', None]}
# cv = GridSearchCV(pipeline, param_grid = parameters, scoring=custom_recall)

#### Changed classifier below ####

In [55]:
##custom recall scorer (changes average from 'binary')
custom_recall = make_scorer(recall_score,average='weighted')

In [57]:
pipeline = Pipeline(
    [('vectorizer',CountVectorizer(tokenizer=tokenize)),
     ('tfidf',TfidfTransformer()),
     ('clf',KNeighborsClassifier())]
)

In [66]:
pipeline.get_params()

{'clf': KNeighborsClassifier(algorithm='auto', leaf_size=15, metric='minkowski',
            metric_params=None, n_jobs=1, n_neighbors=5, p=2,
            weights='uniform'),
 'clf__algorithm': 'auto',
 'clf__leaf_size': 15,
 'clf__metric': 'minkowski',
 'clf__metric_params': None,
 'clf__n_jobs': 1,
 'clf__n_neighbors': 5,
 'clf__p': 2,
 'clf__weights': 'uniform',
 'memory': None,
 'steps': [('vectorizer',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x00000232B5C62268>,
           vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   KNeighborsClassifier(algorithm='auto'

In [67]:
parameters = {'clf__n_neighbors':[3,4,5], 'clf__leaf_size':[5,10,15]}

cv = GridSearchCV(pipeline, param_grid = parameters, scoring=custom_recall)

In [68]:
%time cv.fit(X_train,y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: Recall is

Wall time: 13min 26s


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
       ...owski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__n_neighbors': [3, 4, 5], 'clf__leaf_size': [5, 10, 15]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=make_scorer(recall_score, average=weighted), verbose=0)

In [69]:
cv.best_params_

{'clf__leaf_size': 5, 'clf__n_neighbors': 5}

In [71]:
pipeline.set_params(clf__leaf_size=5, clf__n_neighbors=5)

Pipeline(memory=None,
     steps=[('vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
       ...owski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'))])

In [72]:
%time pipeline.fit(X_train,y_train)

Wall time: 8.58 s


Pipeline(memory=None,
     steps=[('vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
       ...owski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'))])

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [64]:
y_pred = pipeline.predict(X_test)
target_names = y.columns
print(classification_report(y_test, y_pred, target_names=target_names))
print(f'Hamming loss is {hamming_loss(y_test,y_pred)}')

                        precision    recall  f1-score   support

               related       0.83      0.93      0.87      4938
               request       0.66      0.59      0.62      1104
                 offer       0.00      0.00      0.00        26
           aid_related       0.63      0.69      0.66      2714
          medical_help       0.27      0.23      0.25       542
      medical_products       0.24      0.25      0.24       347
     search_and_rescue       0.37      0.05      0.09       192
              security       0.00      0.00      0.00       133
              military       0.72      0.18      0.29       225
           child_alone       0.00      0.00      0.00         0
                 water       0.34      0.44      0.38       430
                  food       0.44      0.54      0.48       731
               shelter       0.38      0.42      0.40       598
              clothing       0.57      0.23      0.33       101
                 money       0.53      

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [73]:
y_pred = pipeline.predict(X_test)
target_names = y.columns
print(classification_report(y_test, y_pred, target_names=target_names))
print(f'Hamming loss is {hamming_loss(y_test,y_pred)}')

                        precision    recall  f1-score   support

               related       0.83      0.93      0.87      4938
               request       0.66      0.59      0.62      1104
                 offer       0.00      0.00      0.00        26
           aid_related       0.63      0.69      0.66      2714
          medical_help       0.27      0.23      0.25       542
      medical_products       0.24      0.25      0.24       347
     search_and_rescue       0.37      0.05      0.09       192
              security       0.00      0.00      0.00       133
              military       0.72      0.18      0.29       225
           child_alone       0.00      0.00      0.00         0
                 water       0.34      0.44      0.38       430
                  food       0.44      0.54      0.48       731
               shelter       0.38      0.42      0.40       598
              clothing       0.57      0.23      0.33       101
                 money       0.53      

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

# START HERE!

In [65]:
y_pred = pipeline.predict(X_test)
target_names = y.columns
print(classification_report(y_test, y_pred, target_names=target_names))

                        precision    recall  f1-score   support

               related       0.83      0.93      0.87      4938
               request       0.66      0.59      0.62      1104
                 offer       0.00      0.00      0.00        26
           aid_related       0.63      0.69      0.66      2714
          medical_help       0.27      0.23      0.25       542
      medical_products       0.24      0.25      0.24       347
     search_and_rescue       0.37      0.05      0.09       192
              security       0.00      0.00      0.00       133
              military       0.72      0.18      0.29       225
           child_alone       0.00      0.00      0.00         0
                 water       0.34      0.44      0.38       430
                  food       0.44      0.54      0.48       731
               shelter       0.38      0.42      0.40       598
              clothing       0.57      0.23      0.33       101
                 money       0.53      

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [19]:
cv.best_params_

{'clf__leaf_size': 15, 'clf__n_neighbors': 2}

In [20]:
pipeline.set_params(clf__leaf_size=15, clf__n_neighbors=2)

Pipeline(memory=None,
     steps=[('vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
       ...owski',
           metric_params=None, n_jobs=1, n_neighbors=2, p=2,
           weights='uniform'))])

In [21]:
pipeline.fit(X_train,y_train)

Pipeline(memory=None,
     steps=[('vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
       ...owski',
           metric_params=None, n_jobs=1, n_neighbors=2, p=2,
           weights='uniform'))])

In [22]:
y_pred = pipeline.predict(X_test)
hamming_loss(y_test,y_pred)

0.07243481379027714

In [41]:
#return to original pipeline
pipeline = Pipeline(
    [('vectorizer',CountVectorizer(tokenizer=tokenize)),
     ('tfidf',TfidfTransformer()),
     ('clf',RandomForestClassifier(random_state=42, n_estimators=100))]
)

In [43]:
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
       ...timators=100, n_jobs=1,
            oob_score=False, random_state=42, verbose=0, warm_start=False))])

# COMPLETE PREDICTOR FUNCTION FOR QUICKER TESTING?

In [79]:
def predictor(string):
    '''
    This function allows for easier testing of the ML pipeline.
    
    input: string
    output: list of predicted labels
    '''
    prediction = pipeline.predict([string])
    

prediction = pipeline.predict(['help'])
print(prediction[0])

[1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]


In [81]:
print(prediction[0])

[1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]


### 9. Export your model as a pickle file

In [74]:
#https://machinelearningmastery.com/save-load-machine-learning-models-python-scikit-learn/
filename = 'model2.pkl'
pickle.dump(pipeline, open(filename, 'wb'))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: ResourceWarning: unclosed file <_io.BufferedWriter name='model2.pkl'>
  This is separate from the ipykernel package so we can avoid doing imports until


### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [85]:
df.head(1)

,id,message,genre,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,direct,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# CREATING PLOTLY PLOT BASED ON NUMBER OF LABELS

In [117]:
label_cols = df.columns[3:]
values = df[label_cols].sum().sort_values(ascending=False)
indices = values.index

In [119]:
values

related                   19906
aid_related               10860
weather_related            7297
direct_report              5075
request                    4474
other_aid                  3446
food                       2923
earthquake                 2455
storm                      2443
shelter                    2314
floods                     2155
medical_help               2084
infrastructure_related     1705
water                      1672
other_weather              1376
buildings                  1333
medical_products           1313
transport                  1201
death                      1194
other_infrastructure       1151
refugees                    875
military                    860
search_and_rescue           724
money                       604
electricity                 532
cold                        530
security                    471
clothing                    405
aid_centers                 309
missing_people              298
hospitals                   283
fire    

In [118]:
indices

Index(['related', 'aid_related', 'weather_related', 'direct_report', 'request',
       'other_aid', 'food', 'earthquake', 'storm', 'shelter', 'floods',
       'medical_help', 'infrastructure_related', 'water', 'other_weather',
       'buildings', 'medical_products', 'transport', 'death',
       'other_infrastructure', 'refugees', 'military', 'search_and_rescue',
       'money', 'electricity', 'cold', 'security', 'clothing', 'aid_centers',
       'missing_people', 'hospitals', 'fire', 'tools', 'shops', 'offer',
       'child_alone'],
      dtype='object')